<a href="https://colab.research.google.com/github/Berto797/Covid_Detector/blob/Inference_Model/COVID_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import cv2
import random
import shutil
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import save_model

In [2]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pwd

/content


# Aplicación de la mascara, separación en subgrupos y guardado


In [ ]:
import os
import zipfile

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data/COVID_data/archive.zip', 'r')
zip_ref.extractall('data/COVID_data')

zip_ref.close()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os

In [ ]:
def mask_apply(image, mask):
  img = cv2.imread(image)
  #print('Shape and dtype Original IMG:\n', img.shape, img.dtype)

  mask = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
  mask_resize = cv2.resize(mask, (img.shape[0],img.shape[1]))
  #print('Shape and dtype Resized MASK:\n', mask_resize.shape, mask_resize.dtype)

  masked_img = cv2.bitwise_and(img,img,mask = mask_resize)

  return masked_img

#masked_img = mask_apply('/content/data/COVID_data/COVID-19_Radiography_Dataset/COVID/images/COVID-1.png', '/content/data/COVID_data/COVID-19_Radiography_Dataset/COVID/masks/COVID-1.png')
#cv2_imshow(masked_img)


In [ ]:
def save_masked_img(img_dir, mask_dir, save_dir):
  for filename in os.listdir(img_dir):
    img_path = os.path.join(img_dir, filename)
    mask_path = os.path.join(mask_dir, filename)

    if os.path.isfile(img_path) and os.path.isfile(mask_path):
        masked_img = mask_apply(img_path, mask_path)
        os.chdir(save_dir)
        cv2.imwrite(filename, masked_img)

# Create directories for COVID, Pneumonia, Lung_Opacity and Normal
Masked_COVID = '/content/data_mask/Masked_Images/COVID'
Masked_Pneumonia = '/content/data_mask/Masked_Images/Viral_Pneumonia'
Masked_LungOpacity = '/content/data_mask/Masked_Images/Lung_Opacity'
Masked_Normal = '/content/data_mask/Masked_Images/Normal'

os.makedirs(Masked_COVID)
os.makedirs(Masked_Pneumonia)
os.makedirs(Masked_LungOpacity)
os.makedirs(Masked_Normal)

# Execute function
save_masked_img('/content/data/COVID_data/COVID-19_Radiography_Dataset/COVID/images', '/content/data/COVID_data/COVID-19_Radiography_Dataset/COVID/masks', Masked_COVID)
save_masked_img('/content/data/COVID_data/COVID-19_Radiography_Dataset/Viral Pneumonia/images', '/content/data/COVID_data/COVID-19_Radiography_Dataset/Viral Pneumonia/masks', Masked_Pneumonia)
save_masked_img('/content/data/COVID_data/COVID-19_Radiography_Dataset/Lung_Opacity/images', '/content/data/COVID_data/COVID-19_Radiography_Dataset/Lung_Opacity/masks', Masked_LungOpacity)
save_masked_img('/content/data/COVID_data/COVID-19_Radiography_Dataset/Normal/images', '/content/data/COVID_data/COVID-19_Radiography_Dataset/Normal/masks', Masked_Normal)


In [ ]:
import random
import shutil

def train_validation_split(source_dir, train_dir, val_dir, not_used_dir, split_size, img_used):
  '''
  Función para separar entre imagenes de entreno, validación y no usadas para el modelo
  Parámetros:
    source_dir --> Directorio de origen con todas las imagenes
    train_dir --> Directorio donde guardar las imágenes de entreno
    val_dir --> Directorio donde guardar las imágenes de validación
    not_used_dir --> Directorio donde guardar las imágenes no utilizadas en el modelo
    split_size --> Tanto por uno que inidca la cantidad de imagenes de entreno sobre el total
    img_used --> Tanto por uno que indica la cantidad total de imagenes utilizadas entre train y val
  '''
  lung_images = os.listdir(source_dir)
  revised_images = []

  for image in lung_images:
    if os.path.getsize(os.path.join(source_dir, image)) == 0:
      print(f"{image} is zero length, so ignoring.")
    else:
      revised_images.append(image)

  rand_revised_lung = random.sample(revised_images, len(revised_images))
  img_used_size = round(len(rand_revised_lung)*img_used)
  train_size = round(img_used_size*split_size)

  for pos in range(len(rand_revised_lung)):
    source_path = os.path.join(source_dir,  rand_revised_lung[pos])
    if pos < train_size:
      destination_path = os.path.join(train_dir, rand_revised_lung[pos])
      shutil.copyfile(source_path, destination_path)
    elif pos >= train_size and pos < img_used_size:
      destination_path = os.path.join(val_dir, rand_revised_lung[pos])
      shutil.copyfile(source_path, destination_path)
    else:
      destination_path = os.path.join(not_used_dir, rand_revised_lung[pos])
      shutil.copyfile(source_path, destination_path)

In [ ]:
import shutil

shutil.rmtree('/content/data_model/Train')
shutil.rmtree('/content/data_model/Validation')
#shutil.rmtree('/content/data_model/Not_Used')

In [ ]:
train_dir = '/content/data_model/Train/'
validation_dir = '/content/data_model/Validation/'
not_used_dir = '/content/data_Not_Used/'

cat_list = ['COVID/', 'Viral_Pneumonia/', 'Lung_Opacity/', 'Normal/']
for cat in cat_list:
  train_cat = os.path.join(train_dir, cat)
  os.makedirs(train_cat)

  val_cat = os.path.join(validation_dir, cat)
  os.makedirs(val_cat)

  not_used_cat = os.path.join(not_used_dir, cat)
  os.makedirs(not_used_cat)

In [ ]:
split_size = 0.8
img_used = 1

train_COVID = '/content/data_model/Train/COVID/'
train_Pneumonia = '/content/data_model/Train/Viral_Pneumonia/'
train_LungOpacity = '/content/data_model/Train/Lung_Opacity/'
train_Normal = '/content/data_model/Train/Normal/'

validation_COVID = '/content/data_model/Validation/COVID/'
validation_Pneumonia = '/content/data_model/Validation/Viral_Pneumonia/'
validation_LungOpacity = '/content/data_model/Validation/Lung_Opacity/'
validation_Normal = '/content/data_model/Validation/Normal/'

not_used_COVID = '/content/data_Not_Used/COVID/'
not_used_Pneumonia = '/content/data_Not_Used/Viral_Pneumonia/'
not_used_LungOpacity = '/content/data_Not_Used/Lung_Opacity/'
not_used_Normal = '/content/data_Not_Used/Normal/'

train_validation_split('/content/data_mask/Masked_Images/COVID', train_COVID, validation_COVID, not_used_COVID, split_size, img_used)
train_validation_split('/content/data_mask/Masked_Images/Viral_Pneumonia', train_Pneumonia, validation_Pneumonia, not_used_Pneumonia, split_size, img_used)
train_validation_split('/content/data_mask/Masked_Images/Lung_Opacity', train_LungOpacity, validation_LungOpacity, not_used_LungOpacity, split_size, img_used)
train_validation_split('/content/data_mask/Masked_Images/Normal', train_Normal, validation_Normal, not_used_Normal, split_size, img_used)

In [ ]:
def train_val_split_batches(source_dir, train_dir, val_dir, split_size, num_batches, cat):
  lung_images = os.listdir(source_dir)
  revised_images = []

  for image in lung_images:
    if os.path.getsize(os.path.join(source_dir, image)) == 0:
      print(f"{image} is zero length, so ignoring.")
    else:
      revised_images.append(image)

  rand_revised_lung = random.sample(revised_images, len(revised_images))
  img_batch = round(len(revised_images)/num_batches)
  left_img = len(revised_images) % num_batches

  for batch in range(1, num_batches+1):
    if batch == num_batches:
      train_size_batch = (img_batch*batch+left_img - img_batch*(batch-1))*split_size + img_batch*(batch-1)
    else:
      train_size_batch = (img_batch*batch - img_batch*(batch-1))*split_size + img_batch*(batch-1)

    for pos in range(img_batch*(batch-1),img_batch*batch+left_img):
      source_path = os.path.join(source_dir,  rand_revised_lung[pos])
      if pos <= train_size_batch:
        batch_path = os.path.join(train_dir, f'batch_{batch}/')
        batch_path = os.path.join(batch_path, f'{cat}/')
        destination_path = os.path.join(batch_path, rand_revised_lung[pos])
        shutil.copyfile(source_path, destination_path)
      else:
        batch_path = os.path.join(val_dir, f'batch_{batch}/')
        batch_path = os.path.join(batch_path, f'{cat}/')
        destination_path = os.path.join(batch_path, rand_revised_lung[pos])
        shutil.copyfile(source_path, destination_path)


In [ ]:
train_dir = '/content/data_model/Train/'
validation_dir = '/content/data_model/Validation/'

num_batches = 4
cat_list = ['COVID/', 'Viral_Pneumonia/', 'Lung_Opacity/', 'Normal/']
for batch in range(1, num_batches+1):
  for cat in cat_list:
    train_batch_path = os.path.join(train_dir, f'batch_{batch}')
    train_batch_cat = os.path.join(train_batch_path, cat)
    os.makedirs(train_batch_cat)

    val_batch_path = os.path.join(validation_dir, f'batch_{batch}')
    val_batch_cat = os.path.join(val_batch_path, cat)
    os.makedirs(val_batch_cat)

In [ ]:
split_size = 0.8
num_batches = 4
train_dir = '/content/data_model/Train/'
validation_dir = '/content/data_model/Validation/'

train_val_split_batches('/content/data_model/Masked_Images/COVID', train_dir, validation_dir, split_size, num_batches, 'COVID')
train_val_split_batches('/content/data_model/Masked_Images/Viral_Pneumonia', train_dir, validation_dir, split_size, num_batches, 'Viral_Pneumonia')
train_val_split_batches('/content/data_model/Masked_Images/Lung_Opacity', train_dir, validation_dir, split_size, num_batches, 'Lung_Opacity')
train_val_split_batches('/content/data_model/Masked_Images/Normal', train_dir, validation_dir, split_size, num_batches, 'Normal')

In [ ]:
import shutil

shutil.make_archive('/content/100img_dataset', 'zip', '/content/data_model/')

'/content/100img_dataset.zip'

In [ ]:
from google.colab import files

files.download('/content/100img_dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Exportado subgrupos de imágenes preparadas para el entreno

In [4]:
import os
import zipfile

#zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data/COVID_data/25perIMG_datasett.zip', 'r')
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data/COVID_data/100img_dataset.zip', 'r')
#zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data/100img_dataset.zip', 'r')
#zip_ref = zipfile.ZipFile('/content/drive/MyDrive/Data/COVID_data/Balanced_dataset.zip', 'r')

zip_ref.extractall('/content/data_model/')

zip_ref.close()

In [ ]:
source_dir = '/content/data_model/Train/Normal/'
train_Normal_dir = '/content/data_model/Used_Normal_Train/'
os.makedirs(train_Normal_dir)
not_used_Normal_dir = '/content/data_model/Not_Used_Normal_Train/'
os.makedirs(not_used_Normal_dir)

images = os.listdir(source_dir)
rand_images = random.sample(images, len(images))

for pos in range(len(rand_images)):
  if pos <= 5500:
    source_path = os.path.join(source_dir,  rand_images[pos])
    destination_path = os.path.join(train_Normal_dir, rand_images[pos])
    shutil.copyfile(source_path, destination_path)
  else:
    source_path = os.path.join(source_dir,  rand_images[pos])
    destination_path = os.path.join(not_used_Normal_dir, rand_images[pos])
    shutil.copyfile(source_path, destination_path)

In [ ]:
images_used = os.listdir('/content/data_model/Used_Normal_Train/')
print(f'Images Normal used: {len(images_used)}')
images_not_used = os.listdir('/content/data_model/Not_Used_Normal_Train/')
print(f'Images Normal not used: {len(images_not_used)}')

Images Normal used: 5501
Images Normal not used: 2653


In [ ]:
import shutil
from google.colab import files

shutil.make_archive('/content/Normal_used', 'zip', '/content/data_model/Used_Normal_Train')
shutil.make_archive('/content/Normal_not_used', 'zip', '/content/data_model/Not_Used_Normal_Train')

files.download('/content/Normal_used.zip')
files.download('/content/Normal_not_used.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Image generator y modelo

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = '/content/data_model/Train/'
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=15,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      zoom_range=0.2)
      #horizontal_flip=True,
      #fill_mode='nearest')

#training_datagen = ImageDataGenerator(rescale = 1./255)

VALIDATION_DIR = '/content/data_model/Validation/'
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(256,256),
  #target_size=(299,299),
	class_mode='categorical',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(256,256),
  #target_size=(299,299),
	class_mode='categorical',
  batch_size=32
)


Found 16933 images belonging to 4 classes.
Found 4232 images belonging to 4 classes.


In [6]:
import os
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model_save = ModelCheckpoint('/content/weights/Model.h5', save_best_only=True, monitor='val_loss', verbose=1)

if not os.path.exists('/content/weights'):
  os.makedirs('/content/weights', mode=0o770, exist_ok=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.85, patience=2, cooldown=1, verbose=1)

lr_schedule = LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch / 20))

## Modelo sin Batch Normalization

In [ ]:
def model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='swish', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3), activation='swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3), activation='swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3), activation='swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3), activation='swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.GlobalAveragePooling2D(),

      tf.keras.layers.Dense(512, activation='swish'),

      tf.keras.layers.Dense(4, activation='softmax')
    ])

  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0014)
  model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  model.summary()

  return model

## Modelo con Batch Normalization. Input size (150, 150, 3)

In [ ]:
def model_BN():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), input_shape=(150, 150, 3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.GlobalAveragePooling2D(),

      tf.keras.layers.Dense(512),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),

      tf.keras.layers.Dense(4, activation='softmax')
    ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  model.summary()

  return model

## Modelo con Batch Normalization. Input size (256, 256, 3)

In [ ]:
def model_BN_training():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(128, (3,3), input_shape=(256, 256, 3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(128, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.GlobalAveragePooling2D(),

      tf.keras.layers.Dense(1024),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),

      tf.keras.layers.Dense(4, activation='softmax')
    ])

  #optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.001209054747596383)
  model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  model.summary()

  return model

In [ ]:
model = model_BN_training()

In [ ]:
history = model.fit(train_generator,
                    epochs=1,
                    verbose=1,
                    validation_data=validation_generator,
                    #callbacks=[lr_schedule])
                    callbacks=[model_save, reduce_lr, early_stopping])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.models import save_model

save_model(model, "/content/model_30_80.h5")

# PREDICCIONES

In [7]:
def model_BN_inference():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(128, (3,3), input_shape=(256, 256, 3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(128, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Conv2D(64, (3,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Dropout(0.2),

      tf.keras.layers.GlobalAveragePooling2D(),

      tf.keras.layers.Dense(1024),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.Activation('swish'),

      tf.keras.layers.Dense(4, activation='softmax')
    ])

  optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
  model.compile(loss = 'categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

  model.summary()

  return model

In [8]:
inference_model = model_BN_inference()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 128)     3584      
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 128)    512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 254, 254, 128)     0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 128)    0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 127, 127, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 128)     1

In [9]:
from tensorflow.keras.models import load_model

#imported_model = load_model('/content/weights/Model_batch1.h5')
#imported_model = load_model('/content/drive/MyDrive/Data/COVID_data/Model_batch1.h5')
#imported_model = load_model('/content/drive/MyDrive/Data/Model.h5')
imported_model = load_model('/content/drive/MyDrive/Data/COVID_data/Model_0.9140.h5')
weights = imported_model.get_weights()

inference_model.set_weights(weights)

In [11]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

imported_model = load_model('/content/drive/MyDrive/Data/COVID_data/Model_0.9140.h5')
weights = imported_model.get_weights()
inference_model.set_weights(weights)

for layer_pos in range(len(imported_model.layers)):
  if isinstance(imported_model.layers[layer_pos], tf.keras.layers.BatchNormalization):
    print(imported_model.layers[layer_pos].name, inference_model.layers[layer_pos].name)
    layer = imported_model.layers[layer_pos]
    saved_mean = layer.moving_mean.numpy()
    saved_variance = layer.moving_variance.numpy()
    saved_gamma = layer.gamma.numpy()
    saved_beta = layer.beta.numpy()

    layer_i = inference_model.layers[layer_pos]
    layer_i.moving_mean.assign(saved_mean)
    layer_i.moving_variance.assign(saved_variance)
    layer_i.gamma.assign(saved_gamma)
    layer_i.beta.assign(saved_beta)
    layer_i.training = False
    inference_model.layers[layer_pos] = layer_i

  if isinstance(inference_model.layers[layer_pos], tf.keras.layers.Dropout):
    inference_model.layers[layer_pos].training = False

"""
for layer in imported_model.layers:
  for layer_i in inference_model.layers:
    if isinstance(layer_i, tf.keras.layers.BatchNormalization):
      print(layer.name)
    #print(layer.name)
    if isinstance(layer, tf.keras.layers.BatchNormalization):
      if isinstance(layer_i, tf.keras.layers.BatchNormalization):
        print(layer.name, layer_i.name)
        saved_mean = layer.moving_mean.numpy()
        saved_variance = layer.moving_variance.numpy()
        saved_gamma = layer.gamma.numpy()
        saved_beta = layer.beta.numpy()

        layer_i.moving_mean.assign(saved_mean)
        layer_i.moving_variance.assign(saved_variance)
        layer_i.gamma.assign(saved_gamma)
        layer_i.beta.assign(saved_beta)

    break
"""

"""
#model = tf.keras.models.clone_model(model)
for layer in inference_model.layers:
    if isinstance(layer, tf.keras.layers.Dropout):
        layer.training = False  # Set dropout rate to 0 to disable dropout during inference
"""
# Accuracy using evaluate
loss, accuracy = inference_model.evaluate(validation_generator)
print(f"Accuracy: {accuracy}")

# Confusion Matrix
y_pred = inference_model.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
print(y_pred_classes)
y_true = validation_generator.classes
print(y_true)

confusion_mtx = confusion_matrix(y_true, y_pred_classes)
print(confusion_mtx)
recall = recall_score(y_true, y_pred_classes, average='weighted')
print("Recall:", recall)
accuracy = accuracy_score(y_true, y_pred_classes)
print("Accuracy:", accuracy)

"""
# Precision, Recall
accuracy = tf.keras.metrics.Accuracy()
recall = tf.keras.metrics.Recall()

accuracy.update_state(y_true, y_pred_classes)
recall.update_state(y_true, y_pred_classes)

accuracy_result = accuracy.result().numpy()
recall_result = recall.result().numpy()

print("Accuracy:", accuracy_result)
print("Recall:", recall_result)
"""

batch_normalization batch_normalization
batch_normalization_1 batch_normalization_1
batch_normalization_2 batch_normalization_2
batch_normalization_3 batch_normalization_3
batch_normalization_4 batch_normalization_4
batch_normalization_5 batch_normalization_5
batch_normalization_6 batch_normalization_6
133/133 [==============================] - 30s 150ms/step - loss: 0.2472 - accuracy: 0.9140
Accuracy: 0.9139886498451233
133/133 [==============================] - 17s 123ms/step
[2 2 0 ... 2 0 1]
[0 0 0 ... 3 3 3]
[[ 105  196  376   46]
 [ 207  316  602   77]
 [ 330  538 1050  120]
 [  43   78  137   11]]
Recall: 0.3501890359168242
Accuracy: 0.3501890359168242


'\n# Precision, Recall\naccuracy = tf.keras.metrics.Accuracy()\nrecall = tf.keras.metrics.Recall()\n\naccuracy.update_state(y_true, y_pred_classes)\nrecall.update_state(y_true, y_pred_classes)\n\naccuracy_result = accuracy.result().numpy()\nrecall_result = recall.result().numpy()\n\nprint("Accuracy:", accuracy_result)\nprint("Recall:", recall_result)\n'